In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin'

In [3]:
%matplotlib inline

In [4]:
states = ['sleep','eating','pooping']
pi = [0.35,0.35,0.3]

In [5]:
state_space = pd.Series(pi,index=states,name='states')
print(state_space)
print(state_space.sum())

sleep      0.35
eating     0.35
pooping    0.30
Name: states, dtype: float64
1.0


In [6]:
q_df = pd.DataFrame(columns=states,index=states)
q_df.loc[states[0]] = [0.4,0.2,0.4]
q_df.loc[states[1]] = [0.45,0.45,0.1]
q_df.loc[states[2]] = [0.45,0.25,0.3]

In [7]:
print(q_df)

        sleep eating pooping
sleep     0.4    0.2     0.4
eating   0.45   0.45     0.1
pooping  0.45   0.25     0.3


In [8]:
q = q_df.values
q.shape

(3, 3)

In [9]:
q

array([[0.4, 0.2, 0.4],
       [0.45, 0.45, 0.1],
       [0.45, 0.25, 0.3]], dtype=object)

In [10]:
q_df.sum(axis=1)

sleep      1.0
eating     1.0
pooping    1.0
dtype: float64

In [11]:
from pprint import pprint 

In [12]:
def _get_markov_edges(Q):
    edges = {}
    for col in Q.columns:
        for idx in Q.index:
            edges[(idx,col)] = Q.loc[idx,col]
    return edges

In [13]:
edges_wts = _get_markov_edges(q_df)
edges_wts

{('sleep', 'sleep'): 0.4,
 ('eating', 'sleep'): 0.45,
 ('pooping', 'sleep'): 0.45,
 ('sleep', 'eating'): 0.2,
 ('eating', 'eating'): 0.45,
 ('pooping', 'eating'): 0.25,
 ('sleep', 'pooping'): 0.4,
 ('eating', 'pooping'): 0.1,
 ('pooping', 'pooping'): 0.3}

In [14]:
G = nx.MultiDiGraph()
G.add_nodes_from(states)

In [15]:
G.nodes

NodeView(('sleep', 'eating', 'pooping'))

In [16]:
for k,v in edges_wts.items():
    tmp_origin,tmp_destination = k[0],k[1]
    G.add_edge(tmp_origin,tmp_destination,wight=v,label=v)

In [17]:
pprint(G.edges(data=True))

OutMultiEdgeDataView([('sleep', 'sleep', {'wight': 0.4, 'label': 0.4}), ('sleep', 'eating', {'wight': 0.2, 'label': 0.2}), ('sleep', 'pooping', {'wight': 0.4, 'label': 0.4}), ('eating', 'sleep', {'wight': 0.45, 'label': 0.45}), ('eating', 'eating', {'wight': 0.45, 'label': 0.45}), ('eating', 'pooping', {'wight': 0.1, 'label': 0.1}), ('pooping', 'sleep', {'wight': 0.45, 'label': 0.45}), ('pooping', 'eating', {'wight': 0.25, 'label': 0.25}), ('pooping', 'pooping', {'wight': 0.3, 'label': 0.3})])


In [18]:
pos = nx.drawing.nx_pydot.graphviz_layout(G,prog='dot')
nx.draw_networkx(G, pos)
edge_labels = {(n1,n2):d['label'] for n1,n2,d in G.edges(data=True)}
nx.draw_networkx_edge_labels(G , pos, edge_labels=edge_labels)
nx.drawing.nx_pydot.write_dot(G, 'pet_dog_markov.dot')

FileNotFoundError: [WinError 2] "dot" not found in path.

In [19]:
hidden_states = ['healthy','sick']
pi = [0.5,0.5]
state_space = pd.Series(pi,index=hidden_states,name='states')
print(state_space)
print(state_space.sum())

healthy    0.5
sick       0.5
Name: states, dtype: float64
1.0


In [20]:
a_df = pd.DataFrame(columns=hidden_states, index=hidden_states)
a_df.loc[hidden_states[0]] = [0.7, 0.3]
a_df.loc[hidden_states[1]] = [0.4, 0.6]

print(a_df)

a = a_df.values
print('\n', a, a.shape, '\n')
print(a_df.sum(axis=1))

        healthy sick
healthy     0.7  0.3
sick        0.4  0.6

 [[0.7 0.3]
 [0.4 0.6]] (2, 2) 

healthy    1.0
sick       1.0
dtype: float64


#### Now we create the emission or observation probability matrix. This matrix is size M x O where M is the number of hidden states and O is the number of possible observable states. 

#### The emission matrix tells us the probability the dog is in one of the hidden states, given the current, observable state. 

In [21]:
observable_states = states

b_df = pd.DataFrame(columns=observable_states, index=hidden_states)
b_df.loc[hidden_states[0]] = [0.2, 0.6, 0.2]
b_df.loc[hidden_states[1]] = [0.4, 0.1, 0.5]

print(b_df)

b = b_df.values
print('\n', b, b.shape, '\n')
print(b_df.sum(axis=1))

        sleep eating pooping
healthy   0.2    0.6     0.2
sick      0.4    0.1     0.5

 [[0.2 0.6 0.2]
 [0.4 0.1 0.5]] (2, 3) 

healthy    1.0
sick       1.0
dtype: float64


In [22]:
hide_edges_wts = _get_markov_edges(a_df)
pprint(hide_edges_wts)

emit_edges_wts = _get_markov_edges(b_df)
pprint(emit_edges_wts)

{('healthy', 'healthy'): 0.7,
 ('healthy', 'sick'): 0.3,
 ('sick', 'healthy'): 0.4,
 ('sick', 'sick'): 0.6}
{('healthy', 'eating'): 0.6,
 ('healthy', 'pooping'): 0.2,
 ('healthy', 'sleep'): 0.2,
 ('sick', 'eating'): 0.1,
 ('sick', 'pooping'): 0.5,
 ('sick', 'sleep'): 0.4}


In [23]:
# create graph object
G = nx.MultiDiGraph()

# nodes correspond to states
G.add_nodes_from(hidden_states)
print(f'Nodes:\n{G.nodes()}\n')

# edges represent hidden probabilities
for k, v in hide_edges_wts.items():
    tmp_origin, tmp_destination = k[0], k[1]
    G.add_edge(tmp_origin, tmp_destination, weight=v, label=v)

# edges represent emission probabilities
for k, v in emit_edges_wts.items():
    tmp_origin, tmp_destination = k[0], k[1]
    G.add_edge(tmp_origin, tmp_destination, weight=v, label=v)
    
print(f'Edges:')
pprint(G.edges(data=True))    



Nodes:
['healthy', 'sick']

Edges:
OutMultiEdgeDataView([('healthy', 'healthy', {'weight': 0.7, 'label': 0.7}), ('healthy', 'sick', {'weight': 0.3, 'label': 0.3}), ('healthy', 'sleep', {'weight': 0.2, 'label': 0.2}), ('healthy', 'eating', {'weight': 0.6, 'label': 0.6}), ('healthy', 'pooping', {'weight': 0.2, 'label': 0.2}), ('sick', 'healthy', {'weight': 0.4, 'label': 0.4}), ('sick', 'sick', {'weight': 0.6, 'label': 0.6}), ('sick', 'sleep', {'weight': 0.4, 'label': 0.4}), ('sick', 'eating', {'weight': 0.1, 'label': 0.1}), ('sick', 'pooping', {'weight': 0.5, 'label': 0.5})])


In [ ]:
pos = nx.drawing.nx_pydot.graphviz_layout(G, prog='neato')
nx.draw_networkx(G, pos)

# create edge labels for jupyter plot but is not necessary
emit_edge_labels = {(n1,n2):d['label'] for n1,n2,d in G.edges(data=True)}
nx.draw_networkx_edge_labels(G , pos, edge_labels=emit_edge_labels)
nx.drawing.nx_pydot.write_dot(G, 'pet_dog_hidden_markov.dot')

In [24]:
obs_map = {'sleeping':0,'eating':1,'pooping':2}
obs = np.array([1,1,2,1,0,1,2,1,0,2,2,0,1,0,1])

In [26]:
inv_obs_map = dict((v,k) for k,v in obs_map.items())
obs_seq = [inv_obs_map[v] for v in obs]


In [28]:
np.column_stack([obs,obs_seq])

array([['1', 'eating'],
       ['1', 'eating'],
       ['2', 'pooping'],
       ['1', 'eating'],
       ['0', 'sleeping'],
       ['1', 'eating'],
       ['2', 'pooping'],
       ['1', 'eating'],
       ['0', 'sleeping'],
       ['2', 'pooping'],
       ['2', 'pooping'],
       ['0', 'sleeping'],
       ['1', 'eating'],
       ['0', 'sleeping'],
       ['1', 'eating']], dtype='<U11')

In [31]:
print(pd.DataFrame(np.column_stack([obs,obs_seq]),columns=['Obs_code', 'Obs_seq']))

   Obs_code   Obs_seq
0         1    eating
1         1    eating
2         2   pooping
3         1    eating
4         0  sleeping
5         1    eating
6         2   pooping
7         1    eating
8         0  sleeping
9         2   pooping
10        2   pooping
11        0  sleeping
12        1    eating
13        0  sleeping
14        1    eating


#### Using the Viterbi algorithm we can identify the most likely sequence of hidden states given the sequence of observations.

In [33]:
pi, a, b, obs  #隐含状态的概率分布，隐含状态的转换矩阵，隐含状态和可观察状态的转移矩阵，观察序列

([0.5, 0.5], array([[0.7, 0.3],
        [0.4, 0.6]], dtype=object), array([[0.2, 0.6, 0.2],
        [0.4, 0.1, 0.5]], dtype=object), array([1, 1, 2, 1, 0, 1, 2, 1, 0, 2, 2, 0, 1, 0, 1]))

In [37]:
nStates = np.shape(b)[0]
nStates

2

In [38]:
T = np.shape(obs)[0]

In [39]:
path = np.zeros(T)

In [40]:
delta = np.zeros((nStates, T))
phi = np.zeros((nStates, T))

In [41]:
delta

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [42]:
phi

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [46]:
obs,pi,b[:,obs[0]]

(array([1, 1, 2, 1, 0, 1, 2, 1, 0, 2, 2, 0, 1, 0, 1]),
 [0.5, 0.5],
 array([0.6, 0.1], dtype=object))

In [50]:
pi * b[:, obs[0]]

array([0.3, 0.05], dtype=object)

In [51]:
delta[:, 0] = pi * b[:, obs[0]]
phi[:, 0] = 0

In [52]:
delta

array([[0.3 , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.05, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ]])

In [53]:
for t in range(1, T):
    for s in range(nStates):
        delta[s, t] = np.max(delta[:, t-1] * a[:, s]) * b[s, obs[t]] 
        phi[s, t] = np.argmax(delta[:, t-1] * a[:, s])
        print('s={s} and t={t}: phi[{s}, {t}] = {phi} ,delta[{s},{t}] = {delta}'.format(s=s, t=t, phi=phi[s, t],delta=delta[s,t]))

s=0 and t=1: phi[0, 1] = 0.0 ,delta[0,1] = 0.126
s=1 and t=1: phi[1, 1] = 0.0 ,delta[1,1] = 0.009
s=0 and t=2: phi[0, 2] = 0.0 ,delta[0,2] = 0.01764
s=1 and t=2: phi[1, 2] = 0.0 ,delta[1,2] = 0.0189
s=0 and t=3: phi[0, 3] = 0.0 ,delta[0,3] = 0.007408799999999999
s=1 and t=3: phi[1, 3] = 1.0 ,delta[1,3] = 0.001134
s=0 and t=4: phi[0, 4] = 0.0 ,delta[0,4] = 0.001037232
s=1 and t=4: phi[1, 4] = 0.0 ,delta[1,4] = 0.0008890559999999999
s=0 and t=5: phi[0, 5] = 0.0 ,delta[0,5] = 0.0004356374399999999
s=1 and t=5: phi[1, 5] = 1.0 ,delta[1,5] = 5.334335999999999e-05
s=0 and t=6: phi[0, 6] = 0.0 ,delta[0,6] = 6.098924159999999e-05
s=1 and t=6: phi[1, 6] = 0.0 ,delta[1,6] = 6.534561599999998e-05
s=0 and t=7: phi[0, 7] = 0.0 ,delta[0,7] = 2.5615481471999993e-05
s=1 and t=7: phi[1, 7] = 1.0 ,delta[1,7] = 3.920736959999999e-06
s=0 and t=8: phi[0, 8] = 0.0 ,delta[0,8] = 3.586167406079999e-06
s=1 and t=8: phi[1, 8] = 0.0 ,delta[1,8] = 3.0738577766399995e-06
s=0 and t=9: phi[0, 9] = 0.0 ,delta[0,9] = 

In [54]:
phi

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0.],
       [0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1.]])

In [55]:
delta

array([[3.00000000e-01, 1.26000000e-01, 1.76400000e-02, 7.40880000e-03,
        1.03723200e-03, 4.35637440e-04, 6.09892416e-05, 2.56154815e-05,
        3.58616741e-06, 5.02063437e-07, 7.37725866e-08, 2.21317760e-08,
        1.59348787e-08, 2.23088302e-09, 9.36970868e-10],
       [5.00000000e-02, 9.00000000e-03, 1.89000000e-02, 1.13400000e-03,
        8.89056000e-04, 5.33433600e-05, 6.53456160e-05, 3.92073696e-06,
        3.07385778e-06, 9.22157333e-07, 2.76647200e-07, 6.63953280e-08,
        3.98371968e-09, 1.91218545e-09, 1.14731127e-10]])

In [60]:
path

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [61]:
path[t+1]

0.0

In [62]:
path[T-1] = np.argmax(delta[:, T-1])
#p('init path\n    t={} path[{}-1]={}\n'.format(T-1, T, path[T-1]))
for t in range(T-2, -1, -1):
    path[t] = phi[int(path[t+1]),[ t+1]]
    #p(' '*4 + 't={t}, path[{t}+1]={path}, [{t}+1]={i}'.format(t=t, path=path[t+1], i=[t+1]))
    print('path[{}] = {}'.format(t, path[t]))
        

path[13] = 0.0
path[12] = 0.0
path[11] = 1.0
path[10] = 1.0
path[9] = 1.0
path[8] = 1.0
path[7] = 0.0
path[6] = 0.0
path[5] = 0.0
path[4] = 0.0
path[3] = 0.0
path[2] = 0.0
path[1] = 0.0
path[0] = 0.0


In [63]:
def viterbi(pi, a, b, obs):
    
    nStates = np.shape(b)[0]
    T = np.shape(obs)[0]
    
    # init blank path
    path = np.zeros(T)
    # delta --> highest probability of any path that reaches state i
    delta = np.zeros((nStates, T))
    # phi --> argmax by time step for each state
    phi = np.zeros((nStates, T))
    
    # init delta and phi 
    delta[:, 0] = pi * b[:, obs[0]]
    phi[:, 0] = 0

    print('\nStart Walk Forward\n')    
    # the forward algorithm extension
    for t in range(1, T):
        for s in range(nStates):
            delta[s, t] = np.max(delta[:, t-1] * a[:, s]) * b[s, obs[t]] 
            phi[s, t] = np.argmax(delta[:, t-1] * a[:, s])
            print('s={s} and t={t}: phi[{s}, {t}] = {phi}'.format(s=s, t=t, phi=phi[s, t]))
    
    # find optimal path
    print('-'*50)
    print('Start Backtrace\n')
    path[T-1] = np.argmax(delta[:, T-1])
    #p('init path\n    t={} path[{}-1]={}\n'.format(T-1, T, path[T-1]))
    for t in range(T-2, -1, -1):
        path[t] = phi[int(path[t+1]), [t+1]]
        #p(' '*4 + 't={t}, path[{t}+1]={path}, [{t}+1]={i}'.format(t=t, path=path[t+1], i=[t+1]))
        print('path[{}] = {}'.format(t, path[t]))
        
    return path, delta, phi

path, delta, phi = viterbi(pi, a, b, obs)
print('\nsingle best state path: \n', path)
print('delta:\n', delta)
print('phi:\n', phi)


Start Walk Forward

s=0 and t=1: phi[0, 1] = 0.0
s=1 and t=1: phi[1, 1] = 0.0
s=0 and t=2: phi[0, 2] = 0.0
s=1 and t=2: phi[1, 2] = 0.0
s=0 and t=3: phi[0, 3] = 0.0
s=1 and t=3: phi[1, 3] = 1.0
s=0 and t=4: phi[0, 4] = 0.0
s=1 and t=4: phi[1, 4] = 0.0
s=0 and t=5: phi[0, 5] = 0.0
s=1 and t=5: phi[1, 5] = 1.0
s=0 and t=6: phi[0, 6] = 0.0
s=1 and t=6: phi[1, 6] = 0.0
s=0 and t=7: phi[0, 7] = 0.0
s=1 and t=7: phi[1, 7] = 1.0
s=0 and t=8: phi[0, 8] = 0.0
s=1 and t=8: phi[1, 8] = 0.0
s=0 and t=9: phi[0, 9] = 0.0
s=1 and t=9: phi[1, 9] = 1.0
s=0 and t=10: phi[0, 10] = 1.0
s=1 and t=10: phi[1, 10] = 1.0
s=0 and t=11: phi[0, 11] = 1.0
s=1 and t=11: phi[1, 11] = 1.0
s=0 and t=12: phi[0, 12] = 1.0
s=1 and t=12: phi[1, 12] = 1.0
s=0 and t=13: phi[0, 13] = 0.0
s=1 and t=13: phi[1, 13] = 0.0
s=0 and t=14: phi[0, 14] = 0.0
s=1 and t=14: phi[1, 14] = 1.0
--------------------------------------------------
Start Backtrace

path[13] = 0.0
path[12] = 0.0
path[11] = 1.0
path[10] = 1.0
path[9] = 1.0
path[

In [64]:
state_map = {0:'healthy', 1:'sick'}
state_path = [state_map[v] for v in path]

(pd.DataFrame()
 .assign(Observation=obs_seq)
 .assign(Best_Path=state_path))

,Observation,Best_Path
0,eating,healthy
1,eating,healthy
2,pooping,healthy
3,eating,healthy
4,sleeping,healthy
5,eating,healthy
6,pooping,healthy
7,eating,healthy
8,sleeping,sick
9,pooping,sick
